<a href="https://colab.research.google.com/github/nairsatish/4540/blob/master/Copy_of_demistifying.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

From article: https://www.analyticsvidhya.com/blog/2020/02/mathematics-behind-convolutional-neural-network/

In [31]:
#importing required libraries
import numpy as np
import pandas as pd
import tqdm as tqdm
from keras.datasets import mnist

#loading dataset
(x_train, y_train), (x_test,y_test) = mnist.load_data()

#selecting a subset of data (200 images)
x_train = x_train[:200]
y=y_train[:200]

X = x_train.T
X = X/255

y.resize((200,1))
y = y.T
#checking value
pd.Series(y[0]).value_counts()


1    26
9    23
0    21
4    21
3    21
7    21
2    20
6    19
8    15
5    13
dtype: int64

In [32]:
#converting into binary classification
for i in range(y.shape[1]):
    if y[0][i] >4:
        y[0][i] = 1
    else:
        y[0][i] = 0

#checking value counts
pd.Series(y[0]).value_counts()

0    109
1     91
dtype: int64

In [33]:
# initializing filter
f=np.random.uniform(size=(3,5,5))
f = f.T

print('Filter 1', '\n', f[:,:,0], '\n')
print('Filter 2', '\n', f[:,:,1], '\n')
print('Filter 3', '\n', f[:,:,2], '\n')

Filter 1 
 [[0.87233134 0.18787556 0.18639263 0.51416072 0.46584328]
 [0.80166462 0.12395707 0.37648343 0.91319478 0.97135482]
 [0.803823   0.37329268 0.38782101 0.26833416 0.79150365]
 [0.90996067 0.03820499 0.37769827 0.50828163 0.45600691]
 [0.09660339 0.88387124 0.78196186 0.7593555  0.10130872]] 

Filter 2 
 [[0.33637767 0.81476963 0.09125322 0.13389002 0.29747035]
 [0.82782628 0.25310917 0.87420212 0.72481684 0.17534675]
 [0.39166597 0.47107484 0.78218228 0.53971981 0.1434986 ]
 [0.31790388 0.32699311 0.89403662 0.88463554 0.71985362]
 [0.16755737 0.55382599 0.13405806 0.0348854  0.84962825]] 

Filter 3 
 [[0.41269959 0.28026489 0.15211149 0.26370371 0.86436662]
 [0.71992771 0.16687693 0.48823687 0.17323525 0.00234341]
 [0.61091672 0.30180737 0.47335326 0.85508478 0.5781113 ]
 [0.11065004 0.38516223 0.11091295 0.12008973 0.85489349]
 [0.4903697  0.00354015 0.2202798  0.67954242 0.08779453]] 



In [34]:
X.shape, y.shape, f.shape

((28, 28, 200), (1, 200), (5, 5, 3))

In [35]:
# Generating patches from images
new_image = []

# for number of images
for k in range(X.shape[2]):
    # sliding in horizontal direction
    for i in range(X.shape[0]-f.shape[0]+1):
        # sliding in vertical direction
        for j in range(X.shape[1]-f.shape[1]+1):
            new_image.append(X[:,:,k][i:i+f.shape[0],j:j+f.shape[1]])
            
# resizing the generated patches as per number of images
new_image = np.array(new_image)
new_image.resize((X.shape[2],int(new_image.shape[0]/X.shape[2]),new_image.shape[1],new_image.shape[2]))
new_image.shape

(200, 576, 5, 5)

In [36]:
# number of features in data set
s_row = X.shape[0] - f.shape[0] + 1
s_col = X.shape[1] - f.shape[1] + 1
num_filter = f.shape[2]

inputlayer_neurons = (s_row)*(s_col)*(num_filter)
output_neurons = 1 

# initializing weight
wo=np.random.uniform(size=(inputlayer_neurons,output_neurons))

In [37]:
# defining the Sigmoid Function
def sigmoid (x):
    return 1/(1 + np.exp(-x))

# derivative of Sigmoid Function
def derivatives_sigmoid(x):
    return x * (1 - x)

In [38]:
# generating output of convolution layer
filter_output = []
# for each image
for i in range(len(new_image)):
    # apply each filter
    for k in range(f.shape[2]):
        # do element wise multiplication
        for j in range(new_image.shape[1]):
            filter_output.append((new_image[i][j]*f[:,:,k]).sum()) 

filter_output = np.resize(np.array(filter_output), (len(new_image),f.shape[2],new_image.shape[1]))

# applying activation over convolution output
filter_output_sigmoid = sigmoid(filter_output)

filter_output.shape, filter_output_sigmoid.shape

((200, 3, 576), (200, 3, 576))

In [39]:
# generating input for fully connected layer
filter_output_sigmoid = filter_output_sigmoid.reshape((filter_output_sigmoid.shape[0],filter_output_sigmoid.shape[1]*filter_output_sigmoid.shape[2]))

filter_output_sigmoid = filter_output_sigmoid.T

# Linear trasnformation for fully Connected Layer
output_layer_input= np.dot(wo.T,filter_output_sigmoid)
output_layer_input = (output_layer_input - np.average(output_layer_input))/np.std(output_layer_input)

# activation function
output = sigmoid(output_layer_input)

In [40]:
#Error
error = np.square(y-output)/2

#Error w.r.t Output (Gradient)
error_wrt_output = -(y-output)

#Error w.r.t sigmoid transformation (output_layer_input)
output_wrt_output_layer_input=output*(1-output)

#Error w.r.t weight
output_wrt_w=filter_output_sigmoid

In [41]:
#delta change in w for fully connected layer
delta_error_fcp = np.dot(output_wrt_w,(error_wrt_output * output_wrt_output_layer_input).T)
lr = 0.01
wo = wo - lr*delta_error_fcp

In [42]:
#Error w.r.t sigmoid output
output_layer_input_wrt_filter_output_sigmoid = wo.T

#Error w.r.t sigmoid transformation
filter_output_sigmoid_wrt_filter_output = filter_output_sigmoid * (1-filter_output_sigmoid)

# cvalculating derivatives for backprop convolution
error_wrt_filter_output = np.dot(output_layer_input_wrt_filter_output_sigmoid.T,error_wrt_output*output_wrt_output_layer_input) * filter_output_sigmoid_wrt_filter_output
error_wrt_filter_output = np.average(error_wrt_filter_output, axis=1)
error_wrt_filter_output = np.resize(error_wrt_filter_output,(X.shape[0]-f.shape[0]+1,X.shape[1]-f.shape[1]+1, f.shape[2]))

In [43]:
filter_update = []
for i in range(f.shape[2]):
    for j in range(f.shape[0]):
        for k in range(f.shape[1]):            
            temp = 0
            spos_row = j
            spos_col = k
            epos_row = spos_row + s_row
            epos_col = spos_col + s_col
            for l in range(X.shape[2]):
                temp = temp + (X[spos_row:epos_row,spos_col:epos_col,l]*error_wrt_filter_output[:,:,i]).sum()
            filter_update.append(temp/X.shape[2])  

filter_update_array = np.array(filter_update)
filter_update_array = np.resize(filter_update_array,(f.shape[2],f.shape[0],f.shape[1]))

In [44]:
for i in range(f.shape[2]):
    f[:,:,i] = f[:,:,i] - lr*filter_update_array[i]

In [ ]:
lst = ['string', 1, 2, 'another string']
for element in lst:
   print type(element)